In [ ]:
import numpy as np 
import pandas as pd 
import nltk
from nltk.corpus import stopwords
import string
import os
import re
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('emails.txt') #read the CSV file

In [ ]:
df.head(100)

In [ ]:
df.shape

In [ ]:
df.spam.value_counts()

In [ ]:
df.columns

In [ ]:
df.drop_duplicates(inplace = True)

In [ ]:
df.shape

In [ ]:
df.spam.value_counts()

In [ ]:
df.isnull().sum()

In [ ]:
from string import punctuation
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import SnowballStemmer

In [ ]:
sent1 = "Subject: why is this"

In [ ]:
sent2 = re.sub("Subject:","",sent1)
sent2

In [ ]:
tokens = word_tokenize(sent1.lower())
tokens

In [ ]:
def clean_txt(sent):    
    #Stripping white spaces before and after the text
    sent = sent.strip()
    #Replacing multiple spaces with a single space
    result = re.sub("\s+", " ", sent)
    result = re.sub("Subject:","",result)
    #Replacing Non-Alpha-numeric and non space charecters with nothing
    result = re.sub("[^\w\s]","",result)
    
    tokens = word_tokenize(result.lower())
    stop_updated = stopwords.words("english")  +  ["would", "could","told"]
    text = [term for term in tokens if term not in stop_updated and len(term) > 2] 
    res = " ".join(text)
    return res

In [ ]:
df.iloc[5000]['text']

In [ ]:
df.iloc[5000]['spam']

In [ ]:
df['ctext'] = df.text.apply(clean_txt)

In [ ]:
df

In [ ]:
df.head(10)

In [ ]:
#Seperate text column and the labels into X and y
X_text = df.ctext.values
y = df.spam.values

In [ ]:
X_text[4000]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_text, y, test_size = 0.20, random_state=3110)
classifier = LogisticRegression()


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_trainmat = tfidf_vectorizer.fit_transform(X_train)

In [ ]:
classifier.fit(X_trainmat.toarray(), y_train)

In [ ]:
X_testmat = tfidf_vectorizer.transform(X_test)
X_testmat

In [ ]:
y_pred = classifier.predict(X_testmat.toarray())

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
accuracy_score(y_test,y_pred)

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
y_predtrain = classifier.predict(X_trainmat)

In [ ]:
accuracy_score(y_train,y_predtrain)

In [ ]:
np.round(classifier.predict_proba(X_testmat),2)

In [ ]:
probabtest = np.round(classifier.predict_proba(X_testmat),2)

In [ ]:
probdf = pd.DataFrame(probabtest)
probdf.loc[10]

In [ ]:
probdf[1].sort_values(ascending = False)

In [ ]:
y_pred_new_threshold = (classifier.predict_proba(X_testmat)[:,1]>=0.5).astype(int)

In [ ]:
classifier.predict_proba(X_testmat)[:,1]

In [ ]:
classifier.predict_proba(X_testmat)[:,1]>=0.8

In [ ]:
(classifier.predict_proba(X_testmat)[:,1]>=0.8).astype(int)

In [ ]:
(np.array([True,False,False,True])).astype(int)

In [ ]:
confusion_matrix(y_test,y_pred_new_threshold)

In [ ]:
accuracy_score(y_test,y_pred_new_threshold)

In [ ]:
probabtest = np.round(classifier.predict_proba(X_testmat),2)

In [ ]:
probabtest

In [ ]:
probdf = pd.DataFrame(probabtest)

In [ ]:
probdf[1].sort_values(ascending = False)

In [ ]:
import pickle

In [ ]:
with open('classifier.pickle','wb') as f:
    pickle.dump(classifier,f)

In [ ]:
with open('tfidfmodel.pickle','wb') as f:
    pickle.dump(tfidf_vectorizer,f)